In [1]:
import os
from getpass import getpass

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [4]:
os.environ["OPENAI_API_KEY"] = getpass()

WEAVIATE_URL = "https://document-vectorstore-ui5n0nz7.weaviate.network"

embeddings = OpenAIEmbeddings()

········


In [20]:
# Load University FAQ data
loader = CSVLoader("./UniFAQdata.csv")
documents = loader.load()

In [12]:
# Split the data into chunks for vectorising
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [13]:
# Create a vectorstore of the data
db = Weaviate.from_documents(docs, embeddings, weaviate_url=WEAVIATE_URL, by_text=False)

In [14]:
# Retrieve top 4 relevant data chunks to provided query
query = "How do I pay my fees?"
docs = db.similarity_search(query, k=4)
docs

[Document(page_content='\ufeffQuestion: How can I pay my fees?\nAnswer: You can pay your fees online by credit or debit card, by bank transfer, by cheque or by recurring card payment. For more details, please visit https://www.ed.ac.uk/finance/students/how-to-pay', metadata={'row': 0, 'source': './UniFAQdata.csv'}),
 Document(page_content='\ufeffQuestion: I am a parent and wish to pay my child’s fees\nAnswer: If you are a parent and wish to pay your child’s fees, you can do so online by credit or debit card, by bank transfer, by cheque or by recurring card payment. You will need to quote your child’s student number and invoice number when making a payment. You can also request a copy of your child’s invoice by contacting the Fees Team at fees@ed.ac.uk with your child’s consent. For more information, please visit https://www.ed.ac.uk/finance/students/how-to-pay/parents-paying-fees', metadata={'row': 29, 'source': './UniFAQdata.csv'}),
 Document(page_content='\ufeffQuestion: Can I pay st

In [17]:
# Define a LLM and a Question-Answer model using the LLM
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.8, 
    verbose=True
)

qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(),
    )

In [18]:
qa.run("How do I pay my fees?")

'You can pay your fees online by credit or debit card, by bank transfer, by cheque, or by recurring card payment. For more details, please visit https://www.ed.ac.uk/finance/students/how-to-pay.'

In [19]:
qa.run("Ive still not recieved an invoice for my fees. What should I do?")

"If you haven't received an invoice for your fees, you should first check your email address and spam folder for any emails from noreply@ed.ac.uk. Additionally, you can check your balance online on MyEd to see if an invoice has been generated for you. If you still haven't received an invoice for your fees, I recommend contacting the Fees Team at fees@ed.ac.uk with your student number and course details. They will be able to assist you further. For more information, you can also visit https://www.ed.ac.uk/finance/students/faqs#FeesandInvoices."